# 네이버 영화평점 감성분석

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [5]:
import pandas as pd

train_df = pd.read_csv('../data/ratings_train.txt', sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [6]:
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [9]:
import re

train_df = train_df.fillna(' ')
#re - 숫자를 공백으로 변경
train_df['document'] = train_df['document'].apply(lambda x : re.sub(r"\d+","",x))
train_df.drop('id', axis=1, inplace=True)            

#Null 및 숫자를 공백으로 변환
test_df = pd.read_csv('../data/ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply(lambda x : re.sub(r"\d+","",x))
test_df.drop('id', axis=1, inplace=True)

In [11]:
from konlpy.tag import Twitter #(Twitter 오류나면 Okt로 바꾸면 됨)

twitter = Twitter()
def tw_tokenizer(text):
    #입력들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = twitter.morphs(text)
    return tokens_ko

tw_tokenizer('오늘은')

['오늘', '은']

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

tfidf_vect = TfidfVectorizer(tokenizer = tw_tokenizer, ngram_range=(1,2),
                            min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

#min_df:최소 빈도수
#ngram_range = (1,1) ==> one, two....
#ngram_range = (1,2) ==> gp back, good time, one, two ...

In [15]:
#Logistic Regression 이용분석 Classification 수행
lg_clf = LogisticRegression(random_state=0)

#Parameter C 최적화-GridSearchCV
params = {'C':[1, 3.5, 4.5, 5.5, 10]}
grid_cv = GridSearchCV(lg_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfidf_matrix_train, train_df['label'])
print(grid_cv.best_params_, round(grid_cv.best_score_,4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 3.5} 0.859


* c값이 3.5일때 86프로로 가장 좋다!

In [18]:
from sklearn.metrics import accuracy_score

#학습 데이터를 적용한 TfidVectorizer를 이용 테스트 데이터를 TF_IDF 값으로 Feature변환
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

#최적 파라미터로 학습된 classifier를 그대로 이용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도:', accuracy_score(test_df['label'],preds))

Logistic Regression 정확도: 0.86192


In [19]:
test_df['document'][150]

'다시 봐도 수작이다. 미국 국수주의라 뭐라는 시점도 있겠으나 전쟁물 중에 손가락안에 꼽을정도로 잘만들어 졌다. 스토리 라인은 뻔하나 그 뻔한걸 너무 잘 살렸다. 언제 봐도 다시봐도 재미 있다. 년이 지난 영화가..'

In [20]:
# 감성분석 결과 0(부정), 1(긍정)
best_estimator.predict(tfidf_vect.transform([test_df['document'][150]]))

array([1], dtype=int64)

In [21]:
text = '여태 보았던 영화중에 제일 재미없네요'
if best_estimator.predict(tfidf_vect.transform([text])) == 0:
    print(f'"{text}" -> 부정일 가능성이 {round(best_estimator.predict_proba(tfidf_vect.transform([text]))[0][0],2)}% 입니다.')
else:
    print(f'"{text}" -> 긍정일 가능성이 {round(best_estimator.predict_proba(tfidf_vect.transform([text]))[0][1],2)}% 입니다.')
#predict_proba는 확률로 나타내는 것

"여태 보았던 영화중에 제일 재미없네요" -> 부정일 가능성이 0.56% 입니다.


In [25]:
text = '최고의 액션 영화입니다.'
if best_estimator.predict(tfidf_vect.transform([text])) == 0:
    print(f'"{text}"-> 부정일 가능성이 {round(best_estimator.predict_proba(tfidf_vect.transform([text]))[0][0],2)}% 입니다.')
else:
    print(f'"{text}"-> 긍정일 가능성이 {round(best_estimator.predict_proba(tfidf_vect.transform([text]))[0][1],2)}% 입니다.')

"최고의 액션 영화입니다."-> 긍정일 가능성이 0.98% 입니다.


In [26]:
result = pd.DataFrame({
    "document": test_df['document'],
    "test": test_df['label'],
    "pred":preds
})

In [27]:
result

,document,test,pred
0,굳 ㅋ,1,1
1,GDNTOPCLASSINTHECLUB,0,0
2,뭐야 이 평점들은.... 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0,0
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,0
4,D만 아니었어도 별 다섯 개 줬을텐데.. 왜 D로 나와서 제 심기를 불편하게 하죠??,0,0
...,...,...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1,1
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0,0
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0,1
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0,0
